In [ ]:
!pip install gym gym[other] tensorflow keras autorom gym[accept-rom-license]

In [ ]:
import numpy as np
from collections import deque
import gym
import random

In [ ]:
env = gym.make("Breakout-v0", obs_type='grayscale', render_mode='rgb_array')
env = gym.wrappers.AtariPreprocessing(env=env, frame_skip=1)
env = gym.wrappers.FrameStack(env=env, num_stack=4)
env = gym.wrappers.RecordVideo(env, '/content/videos', episode_trigger= lambda x : x % 10 == 0)


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (210, 160)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [ ]:
!pip install torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class DQN(nn.Module):
    def __init__(self, action_size):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)  # Assuming input_shape is (channels, height, width)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        # Compute the size of the output of the last conv layer
        def conv2d_size_out(size, kernel_size=3, stride=1):
            return (size - (kernel_size - 1) - 1) // stride + 1

        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(84, 8, 4), 4, 2), 3, 1)
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(84, 8, 4), 4, 2), 3, 1)
        linear_input_size = convw * convh * 64

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(linear_input_size, 512),
            nn.ReLU(),
            nn.Linear(512, action_size)
        )

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        return self.fc(x)

def update_target_network(target, source):
    target.load_state_dict(source.state_dict())


def preprocess_frame(frame):
    # Resize, normalize, etc.
    # ... your preprocessing steps here ...
    processed_frame = np.resize(frame, (84, 84))  # This is a placeholder, replace with actual preprocessing
    return processed_frame

# Initialize frame stacker
frame_stacker = deque(maxlen=4)

# Function to stack frames
def stack_frames(new_frame, is_new_episode=False, stack=frame_stacker):
    frame = preprocess_frame(new_frame)
    
    if is_new_episode:
        # Clear our stack
        stack.clear()
        # Because we're in a new episode, copy the same frame 4x
        for _ in range(4):
            stack.append(frame)
    else:
        # Append frame to deque, automatically removes the oldest frame
        stack.append(frame)
    
    # Stack the frames along the third dimension and return a new numpy array
    stacked_state = np.stack(stack, axis=2)
    return stacked_state

In [ ]:
import copy
# Set parameters
N = 10000  # Replay memory capacity
M = 10000  # Number of episodes
T = 10000  # Max steps per episode
C = 1000  # Target network update frequency
epsilon = 1
epsilon_decay = 0.995
epsilon_min = 0.1
gamma = 0.9
action_size = env.action_space.n  # Number of actions
state_size = env.observation_space.shape[0]  # State size

# Initialize replay memory


Q = DQN(action_size)
Q_hat = copy.deepcopy(Q)




In [ ]:
state = env.reset()
next_state, reward, done, _, _ = env.step(1)
np.asarray(next_state).reshape(84,84,4).shape

(84, 84, 4)

In [ ]:
from tqdm import tqdm

# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Q.to(device)
Q_hat.to(device)
optimizer = optim.Adam(Q.parameters(), lr=0.01)
criterion = nn.MSELoss()
D = deque(maxlen=N)

# Convert numpy array to PyTorch tensor
def preprocess_state(state):
  return torch.tensor(state).float().div(255).unsqueeze(0).to(device)  # Scales to [0,1]


# Training loop
for episode in tqdm(range(M)):
    total_reward = 0
    state = preprocess_state(env.reset()[0])# Add batch dimension
    for t in range(T):
        # Epsilon-greedy action selection
        if np.random.rand() <= epsilon:
            action = random.randrange(action_size)
        else:
            with torch.no_grad():  # No need to track gradients here
                act_values = Q(state)
                action = act_values.max(1)[1].item()  # Choose the action with the highest Q-value

        # Execute action in environment and observe next state and reward
        for _ in range(4):
            next_state, reward, done, _, _ = env.step(action)
            total_reward += reward

        next_state = preprocess_state(next_state)

        # Store transition in D (experience replay buffer)
        D.append((state, action, reward, next_state, done))

        state = next_state

        # Check if the episode is done
        if done :
            #if episode % 100 == 0:
            print(f"Episode: {episode}/{M}, Score: {total_reward}")
            break



        # Train using a random minibatch from D
        if len(D) > 32:
            minibatch = random.sample(D, 32)
            # Extract tensors from the minibatch
            states = torch.cat([s for s, a, r, ns, d in minibatch]).to(device)
            actions = torch.tensor([a for s, a, r, ns, d in minibatch], device=device).long()
            rewards = torch.tensor([r for s, a, r, ns, d in minibatch], device=device).float()
            next_states = torch.cat([ns for s, a, r, ns, d in minibatch]).to(device)
            dones = torch.tensor([d for s, a, r, ns, d in minibatch], device=device).float()


            # Compute Q values for current states
            Q_values = Q(states)
            # Select the Q value for the action taken, which are the ones we want to update
            Q_values = Q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

            # Compute the Q values for next states using the target network
            with torch.no_grad():
                next_state_values = Q_hat(next_states).max(1)[0]
                # If done is true, we want to ignore the next state value
                next_state_values[dones == 1] = 0.0
                # Compute the target Q values
                target_Q_values = rewards + (gamma * next_state_values)

            # Zero the parameter gradients
            optimizer.zero_grad()
            # Compute loss
            loss = criterion(Q_values, target_Q_values)
            # Backward pass
            loss.backward()
            optimizer.step()

    # Update epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Update target network
    if episode % C == 0:
        Q_hat.load_state_dict(Q.state_dict())